In [ ]:
pip install -r requirement.txt

In [ ]:
import json
from kafka import KafkaConsumer
import sys, getopt
import pymongo
from elasticsearch import Elasticsearch
from elasticsearch import helpers

In [ ]:
from config import *

In [ ]:
# Create index if not exist
es = Elasticsearch(config["elasticsearch_uri"])
es.indices.create(index=config["topic"], ignore=[400])
es.indices.put_mapping(
    index=config["topic"],
    body={
        "properties": {
            "cart": {
                "properties": {
                    "articles": {
                        "type": "nested",
                        "properties": {
                            "label": { 
                                "type": "keyword" 
                            },
                            "quantite": { 
                                "type": "integer" 
                            },
                            "prix_unitaire": { 
                                "type": "float" 
                            },
                        }
                    },
                    "prix_total": {
                        "type": "integer"
                    },
                    "nombre_article": {
                        "type": "integer"
                    }
                }
            },
            "client": {
                "properties": {
                    "recordid": {
                        "type": "text"
                    },
                    "fields": {
                        "properties": {
                            "l_adr": {
                                "type": "keyword"
                            },
                            "c_ar": {
                                "type": "integer"
                            }
                        }
                    },
                    "geometry": {
                        "properties": {
                            "type": {
                                "type": "text"
                            },
                            "coordinates": {
                                "type": "geo_point"
                            }
                        }
                    }
                }
            },
            "datetime": {
                "type": "date",
                "format": "strict_date_optional_time"
            }
        }
    }
)

In [ ]:
# Consume topic and upload Data on the index
consumer = KafkaConsumer(config["topic"], bootstrap_servers=config["kafka_uri"])
es = Elasticsearch(config["elasticsearch_uri"])
for message in consumer:
    order = json.loads(message.value.decode())
    print("Order : ")
    print(order)
    print("--------------------")
    es.index(config["topic"],order)

In [ ]:
# Delete Index
# es = Elasticsearch(config["elasticsearch_uri"])
# es.indices.delete(index=config["topic"], ignore=[400])